# Profiling and optimizing Python code
### Code modified from: https://www.youtube.com/watch?v=8qEnExGLZfY


When to optimize?

- Do you need optimization?
  - If speed is not a problem, then there is no reason to optimize
- If yes: Which parts of your code should be optimized?
  - Use a profiler, such as `cProfile`
  - Usually, almost all execution time occurs within a small part of your code
  - Optimize that code, and leave the rest alone
- If you need even better performance
  - Redesign the code completely
  - But this takes effort!

## Example: Find duplicate movie titles

- Read a file of ~10,000 movie titles
- Return a list of movie titles that occur twice
- Search is case insensitive

In [1]:
def read_movies(src) -> list:
    """Read movies from a text file, return the movie titles as a list"""
    
    with open(src) as f:
        movie_list = f.read().splitlines() 
        return movie_list

In [2]:
def is_duplicate(item:str, collection:list) -> bool:
    
    """Determine (True or False) whether a given item (i.e. movie)
       is in a collection of other movie titles (i.e. list)"""
    
    for movie in collection:
        if movie.lower() == item.lower():
            return True
        
    #if you've exhausted the list of movies and found no matches, return False
    return False

In [3]:
def find_duplicate_movies(src='movies.txt') -> list:
    
    """Return all movies that appear twice (i.e. duplicates) in the text file.
       Uses the 2 previous functions."""
    
    movie_list = read_movies(src)
    duplicates = []
    
    while movie_list: #as long as there are still movies to search through
        
        movie = movie_list.pop() #remove the last element of the list, and assign to movie
        
        if is_duplicate(movie, movie_list): #does the movie still occuring the remaining list?
            
            duplicates.append(movie)
            
    
    return duplicates
    

In [4]:
%time duplicate = find_duplicate_movies()

CPU times: user 8.72 s, sys: 31.9 ms, total: 8.75 s
Wall time: 8.8 s


**A whopping 9 seconds** -- Not good.

Ideally at this point we'd like to find where are the bottlenecks in our code. It's very hard to do this as a human (even if you have intuition about where the problem areas may be). So it's better to use a proper tool that can give us a diagnostics report.

## A profiling decorator

- Apply to a function with `@profile`
- Profiles the function using `cProfile`, and prints out a report
- Adapted from the Python 3.6 docs:
  - https://docs.python.org/3/library/profile.html#profile.Profile

A decorator is a function that takes in another function as an argument, and modifies the behavior of that function / adds some additional functionality.

In [5]:
import cProfile, pstats, io


def profile(fnc):
    
    """A decorator that uses cProfile to profile a function. 
       Starts the profile before executing a function, then exeuctes the function,
       then stops the profile, then prints out a diagnostics report.
       
       Lots of boilerplate code from the Python 3 documentation:
       https://docs.python.org/3/library/profile.html#profile.Profile
       """
    
    def inner(*args, **kwargs):
        
        pr = cProfile.Profile()
        pr.enable() ### start the profiler
        
        retval = fnc(*args, **kwargs) ### then actually execute the function
        
        pr.disable() ### then we stop the profiler
        
        ###then print the results to the standard output
        s = io.StringIO()
        sortby = 'cumulative'
        ps = pstats.Stats(pr, stream=s).sort_stats(sortby)
        ps.print_stats()
        print(s.getvalue())
        
        ### then return the actual return value of the inner function we executed
        return retval

    ### execute the innter function
    return inner

--- 

Now let's apply the decorator to the function we'd like to profile!

In [6]:
@profile
def find_duplicate_movies(src='movies.txt'):
    
    """Return all movies that appear twice (i.e. duplicates) in the text file.
       Uses the 2 previous functions."""
    
    movie_list = read_movies(src)
    duplicates = []
    
    while movie_list: #as long as there are still movies to search through
        
        movie = movie_list.pop() #remove the last element of the list, and assign to movie
        
        if is_duplicate(movie, movie_list): #does the movie still occuring the remaining list?
            
            duplicates.append(movie)
            
    
    return duplicates

In [7]:
duplicates = find_duplicate_movies()

         98214041 function calls in 22.621 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.011    0.011   22.621   22.621 <ipython-input-6-4fa0bb3b7aee>:1(find_duplicate_movies)
    10000   12.155    0.001   22.606    0.002 <ipython-input-2-ef9c09082be7>:1(is_duplicate)
 98193766   10.450    0.000   10.450    0.000 {method 'lower' of 'str' objects}
        1    0.000    0.000    0.003    0.003 <ipython-input-1-e2b582f9e247>:1(read_movies)
        1    0.002    0.002    0.002    0.002 {method 'splitlines' of 'str' objects}
    10000    0.001    0.000    0.001    0.000 {method 'pop' of 'list' objects}
        1    0.000    0.000    0.001    0.001 {method 'read' of '_io.TextIOWrapper' objects}
        1    0.000    0.000    0.000    0.000 {built-in method io.open}
        1    0.000    0.000    0.000    0.000 /opt/anaconda3/lib/python3.8/codecs.py:319(decode)
        1    0.000    0.000    0.000    0.000 {buil

The cumulative time column tells us that ~22.6 seconds was spent in the find_duplicate_movies() function.
- Of that time, the majority of it (also ~22.6 seconds) was eaten up by the is_duplicate function.
- So if we want to optimize, then we need to look somewhere in is_duplicate(), and not anywhere else, since the main function by itself doesn't really do that much.
    - Among the ~22.6 seconds, it looks like that around 10 seconds is just spent in the .lower() function of the string.
    - Which was run 98 million times!!!

# Version 2

### Solution: 
- rather than lowercasing the strings every time we want to do the comparison, why don't we just lowercase everything first as soon as we read them in (in the `find_duplicate_movies()` function).
    - likewise, in the `is_duplicate(...)` function, we can just say `if movie == item`
- So now we'll only call the lower function ~10,000 times, rather than 98 million times.

In [8]:
def is_duplicate(item, collection):
    
    """Determine (True or False) whether a given item (i.e. movie)
       is in a collection of other movie titles (i.e. list)"""
    
    for movie in collection:
#         if movie.lower() == item.lower(): ###OLD!!!
        if movie == item: ###NEW!!!

            return True
        
    #if you've exhausted the list of movies and found no matches, return False
    return False

@profile
def find_duplicate_movies(src='movies.txt'):
    
    """Return all movies that appear twice (i.e. duplicates) in the text file.
       Uses the 2 previous functions."""
    
    movie_list = read_movies(src)
    movie_list = [m.lower() for m in movie_list] ### NEW!!!
    duplicates = []
    
    while movie_list: #as long as there are still movies to search through
        
        movie = movie_list.pop() #remove the last element of the list, and assign to movie
        
        if is_duplicate(movie, movie_list): #does the movie still occuring the remaining list?
            
            duplicates.append(movie)
            
    
    return duplicates

In [9]:
duplicates = find_duplicate_movies()

         30276 function calls in 1.347 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.007    0.007    1.347    1.347 <ipython-input-8-5cbffb6c3c5d>:15(find_duplicate_movies)
    10000    1.331    0.000    1.331    0.000 <ipython-input-8-5cbffb6c3c5d>:1(is_duplicate)
        1    0.003    0.003    0.005    0.005 <ipython-input-8-5cbffb6c3c5d>:22(<listcomp>)
        1    0.000    0.000    0.003    0.003 <ipython-input-1-e2b582f9e247>:1(read_movies)
    10000    0.002    0.000    0.002    0.000 {method 'lower' of 'str' objects}
        1    0.002    0.002    0.002    0.002 {method 'splitlines' of 'str' objects}
    10000    0.001    0.000    0.001    0.000 {method 'pop' of 'list' objects}
        1    0.000    0.000    0.001    0.001 {method 'read' of '_io.TextIOWrapper' objects}
        1    0.001    0.001    0.001    0.001 {built-in method io.open}
        1    0.000    0.000    0.000    0.000 /opt/anaconda

**WOW!**
- We reduce the cumulative time down to less than 1.5 seconds!
- like a 10-fold performance improvement.

What else can we do?
- Well, calling the is_duplicate() function multiple times is quite time consuming
- Also , because it's so simple, we can just go ahead and get rid of that function entirely and just put it in the main function.

# Version 3

In [10]:
# def is_duplicate(item, collection):
    
#     """Determine (True or False) whether a given item (i.e. movie)
#        is in a collection of other movie titles (i.e. list)"""
    
#     for movie in collection:
# #         if movie.lower() == item.lower(): ###OLD!!!
#         if movie == item: ###NEW!!!

#             return True
        
#     #if you've exhausted the list of movies and found no matches, return False
#     return False


"""Now we don't even need the is_duplicate function anymore! ^^ """

@profile
def find_duplicate_movies(src='movies.txt'):
    
    """Return all movies that appear twice (i.e. duplicates) in the text file.
       Uses the 2 previous functions."""
    
    movie_list = read_movies(src)
    movie_list = [m.lower() for m in movie_list] 
    duplicates = []
    
    while movie_list: #as long as there are still movies to search through
        
        movie = movie_list.pop() #remove the last element of the list, and assign to movie
        
#         if is_duplicate(movie, movie_list):  ###OLD

        if movie in movie_list:
            
            duplicates.append(movie)
            
    
    return duplicates

**Let's now give it another shot:**

In [11]:
duplicates = find_duplicate_movies()

         20276 function calls in 0.596 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.589    0.589    0.596    0.596 <ipython-input-10-dbdb609c0b10>:18(find_duplicate_movies)
        1    0.002    0.002    0.004    0.004 <ipython-input-10-dbdb609c0b10>:25(<listcomp>)
        1    0.000    0.000    0.003    0.003 <ipython-input-1-e2b582f9e247>:1(read_movies)
    10000    0.002    0.000    0.002    0.000 {method 'lower' of 'str' objects}
        1    0.001    0.001    0.001    0.001 {method 'splitlines' of 'str' objects}
    10000    0.001    0.000    0.001    0.000 {method 'pop' of 'list' objects}
        1    0.000    0.000    0.001    0.001 {method 'read' of '_io.TextIOWrapper' objects}
        1    0.000    0.000    0.000    0.000 {built-in method io.open}
        1    0.000    0.000    0.000    0.000 /opt/anaconda3/lib/python3.8/codecs.py:319(decode)
        1    0.000    0.000    0.000    0.000 {built-

# EVEN BETTER? (BONUS) -- Version 4

**Improving the underlying algorithm**
- At this point we're pretty good, but if we really wanted to be picky, we could use some clever algorithmics to squeeze out even more efficiency.
- We have an exponential / quadratic solution to our problem, because what we're doing is looping through our list of movies, and then for every movie, loop through the list again. So we've got a double loop, so that if our data doubles, then our speed quadruples.
    - The first loop is the list-comprehension, and the second loop is the while loop
- What we want is something a bit more linear


**SOLUTION**:
- Sort things first!

In [12]:
@profile
def find_duplicate_movies(src='movies.txt'):
    
    """Return all movies that appear twice (i.e. duplicates) in the text file.
       Uses the 2 previous functions."""
    
    movie_list = read_movies(src)
    movie_list = [m.lower() for m in movie_list] 
    
    ### ALL BELOW OLD!!
    
#     duplicates = []  
#     while movie_list: #as long as there are still movies to search through       
#         movie = movie_list.pop() #remove the last element of the list, and assign to movie      
# #         if is_duplicate(movie, movie_list):  ###OLD
#         if movie in movie_list:
#             duplicates.append(movie)

    """BETTER SOLUTION BELOW"""
    
    movie_list.sort() #in-place sorting of a python list.
    
    ### if movie titles are the same, they should be consecutive in the list!
    ### zip together the (list of movies minus the last) & the (list of movies minus the first)
    ### so pair the first with the second, the second with the third, etc.
    duplicates = [movie1 for movie1, movie2 in zip(movie_list[:-1], movie_list[1:]) if movie1 == movie2]
            
    
    return duplicates

### Test it in action!

In [13]:
duplicates = find_duplicate_movies()

         10015 function calls in 0.013 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.013    0.013 <ipython-input-12-2e5f3a593210>:1(find_duplicate_movies)
        1    0.005    0.005    0.005    0.005 {method 'sort' of 'list' objects}
        1    0.002    0.002    0.005    0.005 <ipython-input-12-2e5f3a593210>:8(<listcomp>)
        1    0.000    0.000    0.003    0.003 <ipython-input-1-e2b582f9e247>:1(read_movies)
    10000    0.002    0.000    0.002    0.000 {method 'lower' of 'str' objects}
        1    0.002    0.002    0.002    0.002 {method 'splitlines' of 'str' objects}
        1    0.001    0.001    0.001    0.001 <ipython-input-12-2e5f3a593210>:26(<listcomp>)
        1    0.000    0.000    0.001    0.001 {method 'read' of '_io.TextIOWrapper' objects}
        1    0.000    0.000    0.000    0.000 {built-in method io.open}
        1    0.000    0.000    0.000    0.000 /opt/anacond

### Insane!!!!
- So basically, since we've started, we've had over a 1,000x fold performance improvement in our code.

---